In [12]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2

In [2]:
df = pd.read_csv('data/preprocessed_resume.csv', index_col=0)

In [4]:
vectorizer = TfidfVectorizer()
bag = vectorizer.fit_transform(df.text)
bag = bag.toarray()
bag.shape

(2039, 35442)

In [14]:
x = df.drop(['salary', 'text'], axis=1).values.copy()
X = np.concatenate((x, bag), axis=1)
y = df.salary.values
X.shape

(2039, 35454)

**1 подход**

срезаем столбцы с меньшей суммой tf-idf коэффициентов.

In [29]:
ind = np.argpartition(bag.sum(axis=0), kth=(bag.shape[0] - 5000))[-5000:]
bag_5000 = bag[:, ind]
bag_5000.shape

(2039, 5000)

In [30]:
X_cut = np.concatenate((x, bag_5000), axis=1)
X_train, X_test, y_train, y_test = train_test_split(
    X_cut, y, test_size=0.1)
X_cut.shape

(2039, 5012)

In [31]:
regressor_cut = RandomForestRegressor(max_features=2000, n_estimators=1000)
regressor_cut.fit(X_train, y_train)
regressor_cut.score(X_test, y_test)

0.5597183148841436

In [32]:
np.linalg.norm((regressor_cut.predict(X_test) - y_test)) / np.sqrt(y_test.shape[0])

26856.36079043745

In [33]:
np.concatenate((regressor_cut.predict(X_test).reshape(-1, 1), y_test.reshape(-1, 1)), axis=1)[:10]

array([[ 42365.3  ,  80000.   ],
       [ 68869.3  ,  80000.   ],
       [ 60077.   ,  30000.   ],
       [ 68539.399,  45000.   ],
       [ 69295.798,  50000.   ],
       [ 85764.989,  80000.   ],
       [129845.3  , 100000.   ],
       [ 86214.8  ,  75000.   ],
       [ 76228.4  ,  90000.   ],
       [ 45491.   ,  35000.   ]])

In [34]:
np.argsort(regressor_red.feature_importances_)[-20:]

array([3372, 3885, 3503, 3582, 4691, 3432,   11, 4444,    2, 2602, 3202,
       3619,    7, 3213, 3013, 3776, 4100, 3053,    5, 3771])

**2 подход**

находим наиболее важные фичи в tf-idf